# inverterのデータを入力とするLDS

In [6]:
'''
Parameter Estimation for Linear Dynamical Systems
x[t] = A*x[t-1] + w[t]
y[t] = C*x[t] + v[t]
w[t] ~ N(0,Q)
v[t] ~ N(0,R)
'''

import numpy as np
import matplotlib.pyplot as plt
import time

def E_step(y, A, C, Q, R, pi_1, V_1):
    # num of observations
    T = len(y)

    # prediction
    x_p = {} # x_p[t-1] := x_t^{t-1}
    V_p = {} # V_p[t-1] := V_t^{t-1}
    K   = {} # K[t-1]   := K_t
    # filter
    x_f = {} # x_f[t-1] := x_t^t
    V_f = {} # V_f[t-1] := V_t^t
    # smoother
    J   = {} # J[t-1]   := J_{t}
    x_s = {} # x_s[t-1] := x_{t}^T
    V_s = {} # V_s[t-1] := V_{t}^T
    V_a = {} # V_a[t-1] := V_{t,t-1}^T
    # response
    x   = {} # x[t-1]   := {¥hat x}_t
    P   = {} # P[t-1]   := P_t
    P_a = {} # P_a[t-1] := P_{t,t-1}

    # kalman filter
    for t in range(T):
        if t == 0: # initialize
            x_p[t] = pi_1
            V_p[t] = V_1
        else:
            x_p[t] = A * x_f[t-1]
            V_p[t] = A * V_f[t-1] * A.T + Q
        K[t] = V_p[t] * C.T * np.linalg.pinv((C * V_p[t] * C.T + R))
        x_f[t] = x_p[t] + K[t] * (y[t] - C * x_p[t])
        V_f[t] = V_p[t] - K[t] * C * V_p[t]

    # kalman smoother
    x_s[T-1] = x_f[T-1]
    V_s[T-1] = V_f[T-1]
    for t in range(T-1, 0, -1):
        J[t-1] = V_f[t-1] * A.T * np.linalg.pinv(V_p[t])
        x_s[t-1] = x_f[t-1] + J[t-1] * (x_s[t] - A * x_f[t-1])
        V_s[t-1] = V_f[t-1] + J[t-1] * (V_s[t] - V_p[t]) * J[t-1].T

    I = np.mat(np.eye(*A.shape))
    V_a[T-1] = (I - K[T-1] * C) * A * V_f[T-2]
    for t in range(T-1, 1, -1):
        V_a[t-1] = V_f[t-1] * J[t-2].T + J[t-1] * (V_a[t] - A * V_f[t-1]) * J[t-2].T

    # set response
    for t in range(T):
        x[t] = x_s[t]
        P[t] = V_s[t] + x_s[t] * x_s[t].T
        if t == 0: continue
        P_a[t] = V_a[t] + x_s[t] * x_s[t-1].T

    return x, P, P_a

def M_step(y, x, P, P_a):
    # num of observations
    T = len(y)
    # Output matrix
    C = sum([y[t]*x[t].T for t in range(T)]) * np.linalg.pinv(sum([P[t] for t in range(T)]))
    # Output noise covariance
    R = sum([y[t]*y[t].T - C * x[t] * y[t].T for t in range(T)]) / T
    # State dynamics matrix
    A = sum([P_a[t] for t in range(1,T)]) * np.linalg.pinv(sum([ P[t-1] for t in range(1,T)]))
    # State noise covariance
    Q =  (sum([P[t] for t in range(1,T)]) - A * sum([P_a[t] for t in range(1,T)])) / (T - 1)
    #Q = sum( [ (P[t] - A * P_a[t]) for t in range(1,T) ] ) / (T-1)
    # Initail state mean
    pi_1 = x[1]
    # Initial state covariance
    V_1 = P[1] - x[1] * x[1].T

    return A, C, Q, R, pi_1, V_1

def main():
    
    start = time.time()
    
    #テストデータ生成
    """dt = 0.1
    x = np.mat([[0.0],
                [0.0]])
    A = np.mat([[dt,0.0],
                [0.0,1.0]])
    C = np.mat([[1.0,0.0]])
    Q = np.mat([[dt**4/4,0],
                [0,dt**2]])
    R = np.mat([[1.0]])"""
    
    #テストデータ生成(次元を自由に変更可能)(ここの部分を，inverterの出力を保存したファイルを呼び出すプログラムに変える．呼び出した変数は観測変数yに代入)
    #----------------------------------------------------------------------------------------------------------------------------
    dms = 100
    dmo = 100
    dt = 0.1
    x =np.mat(np.zeros([dms, 1]))
    A = np.mat(np.eye(dms, dms))
    C = np.mat(np.eye(dmo, dms))
    Q = np.mat(np.eye(dms, dms))
    R = np.mat(np.eye(dmo, dmo))
    
    #print("A\n%s\nC\n%s\nQ\n%s\nR\n%s" % (A, C, Q, R))
    
    """X = [] # 状態
    Y = [] # 観測
    K = 500 # サンプル数
    for i in range(K):
        #x = A * x + np.mat(np.random.multivariate_normal((0,0),Q)).T
        #x = A * x + np.mat(np.random.multivariate_normal(np.zeros(dms).tolist(),Q)).T
        #y = C * x + np.mat(np.random.normal(0,1)).T
        #y = C * x + np.mat(np.random.normal(np.zeros(dmo).tolist(),R)).T 
        x = A * x + np.mat(np.random.multivariate_normal(np.zeros(dms).tolist(),Q)).T
        y = C * x + np.mat(np.random.multivariate_normal(np.zeros(dmo).tolist(),R)).T
        X.append(x)
        Y.append(y)"""
    #-----------------------------(変更中)
    ind = []
    i = 898
    while(1):
        ind.append(i)
        i = i+1
        if(i > 1200):
            break
    Y=[]
    for j in range(len(ind)):
        img_z = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_input/di11_sry3_sce4_Cririn/img_'+str(ind[j])+'_z.npy')
        img_z_mat = np.mat(img_z)
        Y.append(img_z_mat.T)
    
    #-----------------------------------------------------------------------------------------------------------------------------
    #print("X\n%s\nY\n%s\n" % (X[0].shape, Y[0].shape))
    
    # 推定
    # パラメータの初期値をランダムに振る    
    ums = np.mat(np.eye(dms))
    umo = np.mat(np.eye(dmo))
    
    A_u = np.mat(np.random.rand(dms,dms))
    A = np.multiply(A_u, ums)
    C = np.mat(np.random.rand(dmo,dms))
    
    Q_u = np.mat(np.random.rand(dms,dms))
    Q = np.multiply(Q_u, ums)
    R_u = np.mat(np.random.rand(dmo,dmo))
    R = np.multiply(R_u, umo)

    pi_1 = np.mat(np.random.rand(dms,1))
    V_1_u = np.mat(np.random.rand(dms,dms))
    V_1 = np.multiply(V_1_u, ums)
    
    #print("pi_1\n%s\nV_1\n%s" % (pi_1, V_1))
    old = 0
    diff_old = 100000
    diff_mini = 100000
    eps = 0.01 #収束判定のための閾値
    N = 1000000 # EM回数
    tm = start #start時間をtmに代入
    i = 0
    
    e = E_step(Y, A, C, Q, R, pi_1, V_1)
    for i in range(N):
        ln_like_sums = 0
        m = M_step(Y, *e)
        e = E_step(Y, *m)
        
        A, C, Q, R, pi_1, V_1 = m
        X_hat, pi, pa = e
        
        for t in range(len(ind)):
            #1時刻目(forループの0番目)
            if(t == 0):
                mu_sta = pi_1
                b_sta = np.linalg.det((X_hat[t]-mu_sta).T * np.linalg.pinv(V_1) * (X_hat[t]-mu_sta))
                ln_sta = b_sta - (1/2)*(np.log(np.linalg.det(V_1)))
        
                mu_out = C*X_hat[t]
                b_out = np.linalg.det((Y[t]-mu_out).T * np.linalg.pinv(R) * (Y[t]-mu_out))
                ln_out = b_out - (1/2)*(np.log(np.linalg.det(R)))
        
            #2時刻目以降(forループの1番目以降)
            elif(t > 0):
                mu_sta = A*X_hat[t-1]
                b_sta = np.linalg.det((X_hat[t]-mu_sta).T * np.linalg.pinv(Q) * (X_hat[t]-mu_sta))
                ln_sta = b_sta - (1/2)*(np.log(np.linalg.det(Q)))
        
                mu_out = C*X_hat[t]
                b_out = np.linalg.det((Y[t]-mu_out).T * np.linalg.pinv(R) * (Y[t]-mu_out))
                ln_out = b_out - (1/2)*(np.log(np.linalg.det(R)))
    
            ln_like_sums = ln_like_sums + ln_sta + ln_out
        
        diff = np.abs(old - ln_like_sums)
        if(diff < eps):
            print(i)
            diff = np.abs(old - ln_like_sums)
            print("差異\n%s\n" %diff)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/A_pram.npy', A)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/C_pram.npy', C)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/Q_pram.npy', Q)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/R_pram.npy', R)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/pi_1_pram.npy', pi_1)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/V_1_pram.npy', V_1)
            break
        
        if(i % 5 == 0):
            print (i)
            print("差異\n%s" %diff)
            print("sec/5epoch: %s\n" %(time.time()-tm))
            tm = time.time()
        
        if(diff < diff_old and 20 < i):
            if(diff_mini > diff):
                print (i)
                print("差異\n%s" %diff)
                print("Save\n")
                np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/A_pram.npy', A)
                np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/C_pram.npy', C)
                np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/Q_pram.npy', Q)
                np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/R_pram.npy', R)
                np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/pi_1_pram.npy', pi_1)
                np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/V_1_pram.npy', V_1)
                diff_mini = diff
            else:
                pass
        
        old = ln_like_sums
        diff = diff_old
        i = i + 1
        
    print(N)
    
    # 結果表示
    print("実行時間\n%s" %(time.time()-start))
    print ("A\n%s\nC\n%s\nQ\n%s\nR\n%s\npi_1\n%s\nV_1\n%s" % m)
    """x_hat, pi, pa = e
    # テストデータ
    X1 = []
    X2 = []
    # 推定結果
    X3 = []
    X4 = []
    for x in X:
        X1.append(x[0,0])
        X2.append(x[1,0])
    for i in x_hat:
        X3.append(x_hat[i][0,0])
        X4.append(x_hat[i][1,0])
    plt.plot(X1, 'r-')
    plt.plot(X2, 'b-')
    plt.plot(X3, 'm-')
    plt.plot(X4, 'c-')
    plt.show()"""

if __name__ == '__main__':
    main()

/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:189: RuntimeWarning: divide by zero encountered in log
/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:199: RuntimeWarning: divide by zero encountered in log


0
差異
inf
sec/5epoch: 6.108729600906372



/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:203: RuntimeWarning: invalid value encountered in double_scalars


5
差異
nan
sec/5epoch: 18.769174337387085



/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:195: RuntimeWarning: invalid value encountered in log
/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:185: RuntimeWarning: invalid value encountered in log


10
差異
nan
sec/5epoch: 18.70505928993225



LinAlgError: SVD did not converge

# inverterのデータを入力とするLDS(収束条件変更)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

def E_step(y, A, C, Q, R, pi_1, V_1):
    # num of observations
    T = len(y)

    # prediction
    x_p = {} # x_p[t-1] := x_t^{t-1}
    V_p = {} # V_p[t-1] := V_t^{t-1}
    K   = {} # K[t-1]   := K_t
    # filter
    x_f = {} # x_f[t-1] := x_t^t
    V_f = {} # V_f[t-1] := V_t^t
    # smoother
    J   = {} # J[t-1]   := J_{t}
    x_s = {} # x_s[t-1] := x_{t}^T
    V_s = {} # V_s[t-1] := V_{t}^T
    V_a = {} # V_a[t-1] := V_{t,t-1}^T
    # response
    x   = {} # x[t-1]   := {¥hat x}_t
    P   = {} # P[t-1]   := P_t
    P_a = {} # P_a[t-1] := P_{t,t-1}

    # kalman filter
    for t in range(T):
        if t == 0: # initialize
            x_p[t] = pi_1
            V_p[t] = V_1
        else:
            x_p[t] = A * x_f[t-1]
            V_p[t] = A * V_f[t-1] * A.T + Q
        K[t] = V_p[t] * C.T * np.linalg.pinv((C * V_p[t] * C.T + R))
        x_f[t] = x_p[t] + K[t] * (y[t] - C * x_p[t])
        V_f[t] = V_p[t] - K[t] * C * V_p[t]

    # kalman smoother
    x_s[T-1] = x_f[T-1]
    V_s[T-1] = V_f[T-1]
    for t in range(T-1, 0, -1):
        J[t-1] = V_f[t-1] * A.T * np.linalg.pinv(V_p[t])
        x_s[t-1] = x_f[t-1] + J[t-1] * (x_s[t] - A * x_f[t-1])
        V_s[t-1] = V_f[t-1] + J[t-1] * (V_s[t] - V_p[t]) * J[t-1].T

    I = np.mat(np.eye(*A.shape))
    V_a[T-1] = (I - K[T-1] * C) * A * V_f[T-2]
    for t in range(T-1, 1, -1):
        V_a[t-1] = V_f[t-1] * J[t-2].T + J[t-1] * (V_a[t] - A * V_f[t-1]) * J[t-2].T

    # set response
    for t in range(T):
        x[t] = x_s[t]
        P[t] = V_s[t] + x_s[t] * x_s[t].T
        if t == 0: continue
        P_a[t] = V_a[t] + x_s[t] * x_s[t-1].T

    return x, P, P_a

def M_step(y, x, P, P_a):
    # num of observations
    T = len(y)
    # Output matrix
    C = sum([y[t]*x[t].T for t in range(T)]) * np.linalg.pinv(sum([P[t] for t in range(T)]))
    # Output noise covariance
    R = sum([y[t]*y[t].T - C * x[t] * y[t].T for t in range(T)]) / T
    # State dynamics matrix
    A = sum([P_a[t] for t in range(1,T)]) * np.linalg.pinv(sum([ P[t-1] for t in range(1,T)]))
    # State noise covariance
    Q =  (sum([P[t] for t in range(1,T)]) - A * sum([P_a[t] for t in range(1,T)])) / (T - 1)
    #Q = sum( [ (P[t] - A * P_a[t]) for t in range(1,T) ] ) / (T-1)
    # Initail state mean
    pi_1 = x[1]
    # Initial state covariance
    V_1 = P[1] - x[1] * x[1].T

    return A, C, Q, R, pi_1, V_1

def main():
    
    start = time.time()
    #テストデータ生成(次元を自由に変更可能)(ここの部分を，inverterの出力を保存したファイルを呼び出すプログラムに変える．呼び出した変数は観測変数yに代入)
    #----------------------------------------------------------------------------------------------------------------------------
    dms = 100
    dmo = 100
    dt = 0.1
    """x =np.mat(np.zeros([dms, 1]))
    A = np.mat(np.eye(dms, dms))
    C = np.mat(np.eye(dmo, dms))
    Q = np.mat(np.eye(dms, dms))
    R = np.mat(np.eye(dmo, dmo))"""
    
    #print("A\n%s\nC\n%s\nQ\n%s\nR\n%s" % (A, C, Q, R))
    ind = []
    i = 898
    while(1):
        ind.append(i)
        i = i+1
        if(i > 1200):
            break
    Y=[]
    for j in range(len(ind)):
        img_z = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_input/di11_sry3_sce4_Cririn/img_'+str(ind[j])+'_z.npy')
        img_z_mat = np.mat(img_z)
        Y.append(img_z_mat.T)
    
    #-----------------------------------------------------------------------------------------------------------------------------
    #print("X\n%s\nY\n%s\n" % (X[0].shape, Y[0].shape))
    
    # 推定
    # パラメータの初期値をランダムに振る    
    ums = np.mat(np.eye(dms))
    umo = np.mat(np.eye(dmo))
    
    A = np.mat(np.random.rand(dms,dms))
    #A = np.multiply(A_u, ums)
    C = np.mat(np.random.rand(dmo,dms))
    
    Q_u = np.mat(np.random.rand(dms,dms))
    Q = np.multiply(Q_u, ums)
    R_u = np.mat(np.random.rand(dmo,dmo))
    R = np.multiply(R_u, umo)

    pi_1 = np.mat(np.random.rand(dms,1))
    V_1_u = np.mat(np.random.rand(dms,dms))
    V_1 = np.multiply(V_1_u, ums)
    
    #print("pi_1\n%s\nV_1\n%s" % (pi_1, V_1))
    old = 0
    diff_old = 100000
    diff_mini = 100000
    eps = 0.01 #収束判定のための閾値
    N = 100 # EM回数
    tm = start #start時間をtmに代入
    i = 0
    
    e = E_step(Y, A, C, Q, R, pi_1, V_1)
    for i in range(N):
        ln_like_sums = 0
        ln_sta_sums = 0
        ln_out_sums = 0
        m = M_step(Y, *e)
        e = E_step(Y, *m)
        
        A, C, Q, R, pi_1, V_1 = m
        X_hat, pi, pa = e
        
        for t in range(len(ind)):
            #1時刻目(forループの0番目)
            if(t == 0):
                mu_sta = pi_1
                b_sta = np.linalg.det((X_hat[t]-mu_sta).T * np.linalg.pinv(V_1) * (X_hat[t]-mu_sta))
                ln_sta = b_sta - (1/2)*(np.log(np.linalg.det(V_1)))
        
                mu_out = C*X_hat[t]
                b_out = np.linalg.det((Y[t]-mu_out).T * np.linalg.pinv(R) * (Y[t]-mu_out))
                ln_out = b_out - (1/2)*(np.log(np.linalg.det(R)))
        
            #2時刻目以降(forループの1番目以降)
            elif(t > 0):
                mu_sta = A*X_hat[t-1]
                b_sta = np.linalg.det((X_hat[t]-mu_sta).T * np.linalg.pinv(Q) * (X_hat[t]-mu_sta))
                ln_sta = b_sta - (1/2)*(np.log(np.linalg.det(Q)))
        
                mu_out = C*X_hat[t]
                b_out = np.linalg.det((Y[t]-mu_out).T * np.linalg.pinv(R) * (Y[t]-mu_out))
                ln_out = b_out - (1/2)*(np.log(np.linalg.det(R)))
            
            ln_sta_sums = ln_sta_sums + ln_sta
            ln_out_sums = ln_out_sums + ln_sta
            ln_like_sums = ln_like_sums + ln_sta + ln_out
        print("遷移尤度", ln_sta_sums)
        print("出力尤度", ln_out_sums)
        print("対数尤度", ln_like_sums)
        
        if(i % 5 == 0):
            print (i)
            print("sec/5epoch: %s\n" %(time.time()-tm))
            new_dir_path = '/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)
            os.makedirs(new_dir_path, exist_ok=True)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/A_param.npy', A)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/C_param.npy', C)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/Q_param.npy', Q)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/R_param.npy', R)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/pi_1_param.npy', pi_1)
            np.save('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_'+str(i)+'/V_1_param.npy', V_1)
            tm = time.time()
        
    print(N)
    
    # 結果表示
    print("実行時間\n%s" %(time.time()-start))
    print ("A\n%s\nC\n%s\nQ\n%s\nR\n%s\npi_1\n%s\nV_1\n%s" % m)
    
if __name__ == '__main__':
    main()

/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:158: RuntimeWarning: divide by zero encountered in log
/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:168: RuntimeWarning: divide by zero encountered in log


遷移尤度 55977.1608575
出力尤度 55977.1608575
対数尤度 inf
0
sec/5epoch: 6.980104207992554

遷移尤度 54787.6711489
出力尤度 54787.6711489
対数尤度 inf
遷移尤度 53786.3476355
出力尤度 53786.3476355
対数尤度 inf
遷移尤度 52862.4392494
出力尤度 52862.4392494
対数尤度 inf
遷移尤度 51964.7852552
出力尤度 51964.7852552
対数尤度 inf
遷移尤度 51059.090739
出力尤度 51059.090739
対数尤度 inf
5
sec/5epoch: 21.44430637359619

遷移尤度 50112.98173
出力尤度 50112.98173
対数尤度 inf
遷移尤度 49083.5019343
出力尤度 49083.5019343
対数尤度 inf
遷移尤度 47890.8449352
出力尤度 47890.8449352
対数尤度 inf


/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: overflow encountered in add
/mnt/nfs/takagi/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in add


遷移尤度 nan
出力尤度 nan
対数尤度 nan


LinAlgError: SVD did not converge

In [5]:
pwd

'/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING'

In [7]:
A = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_55/A_param.npy')
C = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_55/C_param.npy')
Q = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_55/Q_param.npy')
R = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_55/R_param.npy')
pi_1 = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_55/pi_1_param.npy')
V_1 = np.load('/mnt/nfs/takagi/Mastercorse_program/Picture_future_world/inverter_LDS/USING/LDS_param/params_55/V_1_param.npy')

In [13]:
V_1

array([[  7.60092020e+273,   2.78953116e+272,  -2.76237897e+273, ...,
         -3.01886211e+272,  -1.08724620e+273,   5.36759103e+273],
       [  2.87706525e+272,   1.86064749e+273,  -2.33267851e+273, ...,
          6.62322455e+272,  -8.95495163e+272,   1.57775254e+273],
       [ -2.77241302e+273,  -2.32106322e+273,   6.40186383e+273, ...,
          4.61539210e+271,   1.85572641e+273,  -5.10228181e+273],
       ..., 
       [ -3.21245291e+272,   6.60911878e+272,   5.81032431e+271, ...,
          5.96383563e+273,   5.28769239e+272,  -1.14909884e+274],
       [ -1.09028219e+273,  -8.94644104e+272,   1.85961940e+273, ...,
          5.24106265e+272,   8.28846472e+272,  -2.75233704e+273],
       [  5.47780272e+273,   1.58019949e+273,  -5.17898506e+273, ...,
         -1.14781298e+274,  -2.77333062e+273,   3.65198158e+274]])

In [15]:
cp -r LDS_param/params_55 /opt/pfw/takagi/LDS_param

In [17]:
cp LDS_sammple.py /opt/pfw/takagi